In [5]:
#Jasmine Widgery
#Project 3
#Fall 2023

In [6]:
#Imports
import pandas
import numpy
import math
import random
import scipy.linalg

import warnings

#suppress warnings
warnings.filterwarnings('ignore') #this suppresses a warning from numpy.exp() that complains about the input being too large.

In [7]:
#Step 1. (a) - use linear regression to calculate w

#Read in training data

#Note-- I prepended the training data in the Excel sheet with a column of 1s
trainDF = pandas.read_excel("Project3.xlsx")
#print(trainDF)

X = numpy.array(trainDF[["X0","Midterm", "Homework", "Quiz"]])
Y = numpy.array(trainDF["Course Grade"])
#print(X)
#print(Y)


XTranspose = numpy.transpose(X)
A = numpy.dot(XTranspose, X)
b = numpy.dot(XTranspose, Y)

w = numpy.dot(numpy.linalg.inv(A), b)
#print(w)

#Step 1. (b) - use found w to compute predicted grades for testing data

#Read in testing data
#Note-- I prepended the testing data in the Excel sheet with a column of 1s
testDF_lin = pandas.read_excel("Project3.xlsx", sheet_name = "Predict")


#estimate scores for each point
predictedGrades_lin = []
for i in testDF_lin.index:
    grade = (w[0] * testDF_lin["X0"][i]) + (w[1] * testDF_lin["Midterm"][i]) + (w[2] * testDF_lin["Homework"][i]) + (w[3] * testDF_lin["Quiz"][i])
    predictedGrades_lin.append(grade)

predictedGrades_lin = numpy.array(predictedGrades_lin)
#testDF_lin["Predicted Grades-linear regression"] = predictedGrades_lin
#print(testDF_lin)


In [8]:
#Step 2. (a) - Calculate course pass/fail for training set

pf = []
for i in trainDF.index:
    if trainDF["Course Grade"][i] < 70:
        trainDF["Course Grade"][i] = -1
        pf.append(-1)
    else:
        pf.append(1)
        trainDF["Course Grade"][i] = 1
        
pf = numpy.array(pf)
Y = numpy.array(trainDF["Course Grade"])

#Step 2. (b) - Run logistic regression algorithm to compute w
def CloseToZero(g):
    numClose = 0
    for i in range(0, len(g)):
        if math.isclose(g[i], 0, abs_tol = 0.01):
            #print("G[i] = " + str(g[i]))
            numClose += 1  
                   
    if numClose == len(g):
        return True
    else:
        return False
N = len(Y)
MAX = N - 1
global wFinal
global numIter
numIter = 0
def logisticRegressionStochastic(w, eta):
    global wFinal
    global numIter
    #choose random D(x, y)
    index = random.randint(0, MAX)
    randX = X[index]
    randX = numpy.array(randX)
    randY = Y[index]
    #print("Point: " + str(randX) + " , " + str(randY))
    
    num = -randY * randX
    numToExp = randY * numpy.dot(w, randX)
    denom = 1 + numpy.exp(numToExp)
    gradient = num / denom
    gradient = numpy.array(gradient)
    
    wNext = w - (eta * gradient)
    
    #print("Gradient: " + str(gradient))
    #print("W: " + str(wNext))
    if CloseToZero(gradient) or numIter > 2200:
        wFinal = wNext
        print("Final w is: " + str(wNext))
        #print("Final g is: " + str(gradient))
        print("NumIter: " + str(numIter))
        return wNext
    else:
        numIter += 1
        logisticRegressionStochastic(wNext, eta)

w = [0, 0, 0, 0]                       
logisticRegressionStochastic(w, 0.00008)

def sigmoid(x):
    return 1 / (1 + numpy.exp(-x))



#Step 2. (c) - using found weights, calculate probability for each point of passing the course
w = wFinal
probs = []
for i in testDF_lin.index:
    prob = (w[0] * testDF_lin["X0"][i]) + (w[1] * testDF_lin["Midterm"][i]) + (w[2] * testDF_lin["Homework"][i]) + (w[3] * testDF_lin["Quiz"][i])
    prob = sigmoid(prob)
    probs.append(prob)
    
probs = numpy.array(probs)
testDF_lin["Probabilities - logistic regression"] = probs
print(testDF_lin)


Final w is: [-0.0048891  -0.05344858  0.01489385  0.07413073]
NumIter: 2201
    X0  Midterm  Homework  Quiz  Probabilities - logistic regression
0    1       49        79    71                             0.978459
1    1       56        21    46                             0.673658
2    1       58       100    79                             0.985808
3    1       61        82    58                             0.905135
4    1       62        90    62                             0.932002
5    1       66        99    75                             0.970739
6    1       71        73    65                             0.891486
7    1       73        87    80                             0.965095
8    1       73        97    73                             0.950245
9    1       74        73    77                             0.944550
10   1       74        85    66                             0.900117
11   1       74       100    87                             0.981633
12   1       78        79  